In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nfl2022-input-data/data/validation_input_data.csv
/kaggle/input/nfl2022-input-data/data/simulations/Game2020122713Play1035
/kaggle/input/nfl2022-input-data/data/figures/decoder_variables.pkl
/kaggle/input/nfl2022-input-data/data/figures/attention.pkl
/kaggle/input/nfl2022-input-data/data/figures/static_variables.pkl
/kaggle/input/nfl2022-input-data/data/figures/encoder_variables.pkl
/kaggle/input/nfl2022-input-data/data/evaluations/yards_sim_vs_real.pkl
/kaggle/input/nfl-big-data-bowl-2022/players.csv
/kaggle/input/nfl-big-data-bowl-2022/PFFScoutingData.csv
/kaggle/input/nfl-big-data-bowl-2022/tracking2019.csv
/kaggle/input/nfl-big-data-bowl-2022/tracking2020.csv
/kaggle/input/nfl-big-data-bowl-2022/games.csv
/kaggle/input/nfl-big-data-bowl-2022/tracking2018.csv
/kaggle/input/nfl-big-data-bowl-2022/plays.csv


This notebook is my submission for the NFL Big Data Bowl 2022. 

The recently (2019) developed temporal fusion transformer (TFT) uses a multidimensional time series as input to forecast the multidimensional future. In this project, it predicts the next coordinates, speed, acceleration, direction, orientation, and distance of all 22 players in kickoff returns given the past 4 frames (0.8 secs) as input. By iteratively using the network's predictions as future input, one can continuously predict the next frame (dt = 0.2 secs) and simulate kickoff returns until the returner is tackled, runs out of bounds, or scores a touchdown.

Using a trained TFT, I ranked players based on hypothetical scenarios. A ranking system based solely on historical data is unnormalized in that teams and players face different teams and lineups in a given year increasing the factor of luck into rankings. My rankings are computed based on testing teams and players against the same set of conditions, i.e. the lineup of the most frequent player at each position, and provide a decomposition based on how well each returner performs with their special teams lineup as well as out-of-system performances when returners are substituted into other teams' special teams lineups.

The tracking data is processed to a set of variables, some varying during each play (e.g. position) and some staying static, the same, (e.g. returner's weight), that represent a time instance. The aforementioned variables the network predicts are also considered as time varying input variables and are added to the input to the network when they precede the current time index. The static variables are as follows for each of the 22 players:

1. Weight
2. Height
3. Non-special teams position
4. 40 yd dash
5. Vertical jump
6. Bench press
7. Broad jump
8. 3Cone
9. Shuttle run
10. Draft pick number
11. Returner index
12. Kickoff team name
13. Returning team name
14. Play direction
15. Kicking team home or away.

Some additional pre-processing steps were applied. The variables were transformed so that the kickoff occurs on the right-hand side of the field to the left-hand side endzone. If combine data was not available for a player, the average score for that position was assigned to the player. Before all variables are input to the network, the players' names are sorted in increasing y-direction for the kicking team and increasing x-direction, then increasing y-direction for the receiving team.